In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import pickle
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten, Conv1D, MaxPooling1D, Dropout
from keras.utils import to_categorical

In [3]:
df = pd.read_csv('articles.csv')

In [4]:
texts = df['title'].tolist()
labels = df['category'].tolist()
print(labels)

['poder', 'ilustrada', 'poder', 'ilustrada', 'mercado', 'mundo', 'poder', 'esporte', 'poder', 'tec', 'poder', 'mundo', 'mercado', 'mercado', 'mercado', 'cotidiano', 'esporte', 'mundo', 'mercado', 'poder', 'poder', 'poder', 'mercado', 'cotidiano', 'ilustrada', 'esporte', 'mercado', 'ambiente', 'mercado', 'esporte', 'ilustrada', 'cotidiano', 'mercado', 'esporte', 'poder', 'ilustrada', 'mercado', 'cotidiano', 'mundo', 'cotidiano', 'equilibrioesaude', 'esporte', 'esporte', 'sobretudo', 'mundo', 'cotidiano', 'colunas', 'poder', 'mercado', 'mercado', 'cotidiano', 'mercado', 'educacao', 'esporte', 'mercado', 'poder', 'mundo', 'mercado', 'poder', 'colunas', 'mercado', 'poder', 'colunas', 'colunas', 'ilustrada', 'tv', 'banco-de-dados', 'poder', 'mercado', 'poder', 'cotidiano', 'colunas', 'colunas', 'colunas', 'mundo', 'ilustrada', 'opiniao', 'colunas', 'colunas', 'poder', 'colunas', 'ciencia', 'colunas', 'colunas', 'mercado', 'esporte', 'mercado', 'colunas', 'colunas', 'poder', 'cotidiano', 'pa

In [5]:
max_words = 10000

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
with open('saved_tokenizer/tokenizer.json', 'wb') as f:
    pickle.dump(tokenizer, f)

In [6]:
max_length = 250
data = pad_sequences(sequences, maxlen=max_length)

le = LabelEncoder()
labels_encoded = le.fit_transform(labels)
labels_onehot = to_categorical(labels_encoded, num_classes=48)
# Salvar o LabelEncoder
with open('saved_models/label_encoder.pkl', 'wb') as f:
    pickle.dump(le, f)

# Salvar as classes do LabelEncoder
np.save('saved_models/label_encoder_classes.npy', le.classes_)
print(labels_encoded)
print(len(labels_encoded))
print(labels_onehot)
print(len(labels_onehot))

[35 23 35 ... 18 29 12]
167053
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
167053


In [7]:
label_names = le.classes_
##
# idx = np.argsort(label_names)
# label_names = label_names[idx]
# labels_encoded = labels_encoded[idx]
# print(label_names)
# print(labels_encoded)
# print(len(labels_encoded))

In [8]:
# print(X_train.shape)
# print(y_train.shape)
# print(X_val.shape)
# print(y_val.shape)
# print(X_test.shape)
# print(y_test.shape)

In [9]:
num_rows_titles = df['title'].shape[0]
num_rows_categories = df['category'].shape[0]

if num_rows_titles == num_rows_categories:
    print('A quantidade de linhas de títulos e categorias é igual.')
else:
    print('A quantidade de linhas de títulos e categorias é diferente.')

A quantidade de linhas de títulos e categorias é igual.


In [10]:
print(df['category'].isna().sum())

0


In [11]:
# X_train, X_test, y_train, y_test = train_test_split(data, labels_encoded, test_size=0.2, random_state=42)
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
# 
# ## Configuracao de Rede Neural Densa
# model = Sequential([
#     Embedding(max_words, 50, input_length=max_length),
#     Flatten(),
#     Dense(128, activation='relu'),
#     Dense(64, activation='tanh'),
#     Dense(32, activation='sigmoid'),
#     Dense(48, activation='softmax')
# ])
# 
# ## Compile modelo Rede neural densa
# model.compile(loss='sparse_categorical_crossentropy',
#               optimizer='adam',
#               metrics=['accuracy'])
# 
# ## treino Rede densa
# model_trained_dense = model.fit(X_train, y_train,
#           epochs=10,
#           batch_size=32,
#           validation_data=(X_val, y_val))
# 
# loss, accuracy = model_trained_dense.evaluate(X_test, y_test, batch_size=32)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(data, labels_onehot, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

## Configurcao de CNN
model_CNN = Sequential([
    Embedding(max_words, 50, input_length=max_length),
    Conv1D(128, 5, activation='relu'),
    MaxPooling1D(5),
    Dropout(0.2),
    Conv1D(64, 5, activation='tanh'),
    MaxPooling1D(5),
    Dropout(0.2),
    Conv1D(32, 5, activation='sigmoid'),
    MaxPooling1D(5),
    Dropout(0.2),
    Flatten(),
    Dense(48, activation='softmax')
])

## Compile CNN
model_CNN.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

model_trained_cnn = model_CNN.fit(X_train, y_train,
          epochs=5,
          batch_size=64,
          validation_data=(X_val, y_val))

loss_cnn, accuracy_cnn = model_CNN.evaluate(X_test, y_test, batch_size=64)

Epoch 1/5
1671/1671 [==============================] - 111s 64ms/step - loss: 2.5708 - acc: 0.1658 - val_loss: 2.2862 - val_acc: 0.2599
Epoch 2/5
1671/1671 [==============================] - 86s 51ms/step - loss: 2.2141 - acc: 0.2928 - val_loss: 2.0791 - val_acc: 0.3738
Epoch 3/5
1671/1671 [==============================] - 79s 47ms/step - loss: 1.9347 - acc: 0.4545 - val_loss: 1.7849 - val_acc: 0.5348
Epoch 4/5
1671/1671 [==============================] - 79s 48ms/step - loss: 1.7312 - acc: 0.5405 - val_loss: 1.6198 - val_acc: 0.5857
Epoch 5/5
523/523 [==============================] - 6s 11ms/step - loss: 1.5425 - acc: 0.6074


In [13]:
# print("Rede densa loss:")
# print(loss)
# print("Rede densa accuracy:")
# print(accuracy)
print("CNN loss:")
print(loss_cnn)
print("CNN accuracy:")
print(accuracy_cnn)

model_CNN.save('saved_models/model.h5')
with open('saved_models/model.pkl', 'wb') as f:
    pickle.dump(model_CNN, f)

CNN loss:
1.5424906015396118
CNN accuracy:
0.6074346899986267
